# Weighted Mean Analysis

To prove a point

In [1]:
import sys
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/standard_modules')
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project')

from collation import collator, hvc_snapshot as snap, collation_tools as ct
from synthesis import hvc_looper as hvcl, magnetic_field_derivation as mfd, KStest, calculate as calc, weighted_mean as wgm
from astropy.table import vstack

collated_data = collator.data_whole_sky(False, load_data=["../data_processed/proc_rms","../data_processed/proc_hvcs"], h1_img="../data_catalog/hi4pi-hvc-nhi-car.fits", override_RMs=True)

master_hvcs = hvcl.load_HVCs(collated_data, hvc_indicies=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 24])

=== WHOLE-SKY DATA COLLATION ===
Gathering data ...
Getting H-alpha emission
Extracting RMs
Getting HVC location data
Getting HI emission
Interpolating
Collation complete
=== HVC RM LOADER ===
Taking HVC snapshots
Process complete


In [2]:
def get_stacked_sets(snapshots):
    inners = []
    outers = []
    for hvc_snap in snapshots:
        inner_rms, outer_rms = KStest.split_RMs(hvc_snap["RMs"],hvc_snap["HVC"]["SkyCoord"], KStest.morph_ring(hvc_snap))
        inners.append(inner_rms)
        outers.append(outer_rms)

    master_rm_inner = vstack(inners)
    master_rm_outer = vstack(outers)

    return master_rm_inner, master_rm_outer, inners, outers

master_rm_inner, master_rm_outer, inners, outers = get_stacked_sets(master_hvcs)

In [3]:
import numpy as np
from astropy.table import Table

def weighted_average_individual(data, uncs):
    weights = 1 / (uncs)**2

    return np.average(data, weights=weights), 1/np.sum(weights)

def weighted_average(inners, outers):
    avg_in = []
    unc_in = []
    avg_ot = []
    unc_ot = []
    for i in range(len(inners)):
        av, un = weighted_average_individual(inners[i]["B_virtual [int]"], inners[i]["B_virtual_unc [int]"])
        avg_in.append(av)
        unc_in.append(un)
        av, un = weighted_average_individual(outers[i]["B_virtual [int]"], outers[i]["B_virtual_unc [int]"])
        avg_ot.append(av)
        unc_ot.append(un)
    
    avg_diff = np.array(avg_in) - np.array(avg_ot)
    unc_diff = np.sqrt(np.array(unc_in) ** 2 + np.array(unc_ot) ** 2)

    t = Table()
    t["Average [inner]"] = np.array(avg_in)
    t["Average [outer]"] = np.array(avg_ot)
    t["Average [diff]"] = avg_diff
    t["Avg Unc [inner]"] = np.array(unc_in)
    t["Avg Unc [outer]"] = np.array(unc_ot)
    t["Avg Unc [diff]"] = unc_diff

    return t

wavgs = wgm.weighted_average(inners, outers)
wavgs

Average [inner],Average [outer],Average [diff],Avg Unc [inner],Avg Unc [outer],Avg Unc [diff]
float64,float64,float64,float64,float64,float64
-9.010901261901203,-16.691518833687702,7.680617571786499,0.1544710566244403,0.06975604516150032,0.16949104156634448
-0.6960604023293531,-11.978620978240315,11.282560575910962,0.4934626372172539,0.03289730408607167,0.4945579914889038
13.132260146632955,0.8190631739611469,12.313196972671808,0.20749211378340493,0.06703715396383249,0.2180526479863889
-7.30229848617082,-3.978661824845781,-3.3236366613250388,1.1564596550972988,0.048370593054644674,1.1574707979643466
-2.168078601732155,-6.370877536021758,4.202798934289603,0.20314919434020826,0.04004602282589842,0.2070586368767265
-3.7630650039131255,-2.4956028506573738,-1.2674621532557517,0.2547830594711783,0.07201239105270783,0.26476440821723396
-11.764841199688588,-17.481018590675696,5.716177390987108,0.561224497952308,0.14982108400037036,0.5808780373820878
12.994898045135406,2.6170295774063628,10.377868467729042,0.46755894063459996,0.17864583221397004,0.5005254202683129
1.6847642877788824,8.566817538848705,-6.882053251069822,0.627066596774432,0.04772657752967211,0.628880229449904


In [4]:
for i in range(len(master_hvcs)):
    print(master_hvcs[i]["HVC"]["Name"])
    print(calc.B_virt(master_hvcs[i]["HVC"]["NH"], master_hvcs[i]["HVC"]["e_NH"], wavgs[i]["Average [diff]"])[0].n)

G038.7-37.3-241
1.1674538475147485
G038.8-33.2-261
1.1282560140867086
G043.4-30.4-230
1.6134534179160942
G089.0-64.7-311
-0.6647273066338089
G133.5-75.6-294
0.7605065031791882
G248.9+36.8+181
-0.34402543686490367
G261.5-34.5+237
3.39398037916118
G262.4-26.2+143
3.9435900254244936
G264.6-53.3+224
-2.1793169469598506
G267.1-38.4+263
3.4833753468970174
G271.6-48.4+297
0.07668164585606313
G271.7-53.1+254
-0.017548541740356634
G323.0+25.6+122
0.30487844158483235


# Mixing it with the other two

In [5]:
import sys
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/standard_modules')
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project')

from collation import collator, hvc_snapshot as snap, collation_tools as ct
from synthesis import hvc_looper as hvcl, magnetic_field_derivation as mfd, KStest, uncertainty_subtraction as us

collated_data = collator.data_whole_sky(False, load_data=["../data_processed/proc_rms","../data_processed/proc_hvcs_filtered"], h1_img="../data_catalog/hi4pi-hvc-nhi-car.fits", override_RMs=True)

#master_hvcs = hvcl.manual_filter_HVCs(collated_data, hvc_indicies=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 16, 22, 24])
#master_hvcs = hvcl.load_HVCs(collated_data, [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 24], "../data_processed/hvc_rms/")

#results = hvcl.uncertainty_subtract_HVCs(collated_data, filter_significant=False, hvc_indicies=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 16, 22, 24])#, save_file="", filter_significant=False)

results = ct.read_processed("../data_processed/results_pre_RM")
results_filtered = us.filter_significant(results)

#collated_data['HVCs']
results

=== WHOLE-SKY DATA COLLATION ===
Gathering data ...
Getting H-alpha emission
Extracting RMs
Getting HVC location data
Getting HI emission
Interpolating
Collation complete


Name,Statistic,p_value,Statistic_x,Statistic_sgn,Statistic_diff,Significant,Sigma [meas],Sigma [obsv],Sigma [true],Sigma [diff],FWHM
str15,float64,float64,float64,int32,float64,bool,float64,float64,float64,float64,float64
G038.7-37.3-241,0.2074738871132492,3.5194997048780594e-09,-23.129523665567874,-1,16.86046632442212,True,7.315711919790323,21.44495684338161,20.158534994417067,-7.5559356070774975,-17.792868626509186
G038.8-33.2-261,0.31066825775656326,1.3943532792837091e-08,-6.963302929232215,-1,12.50616654023725,True,7.247502115395694,22.736827410419288,21.5508012328667,-3.6485845077294528,-8.591759934790694
G043.4-30.4-230,0.24109469074986317,3.606854288675973e-11,-9.607423653882606,-1,17.56975352329457,True,7.411948080584831,26.053745901546854,24.977203629571594,-2.3590579203581576,-5.555156878248415
G089.0-64.7-311,0.3738140417457306,0.015075834341180974,-17.51573024976091,1,-12.06027479430546,False,4.475548858546416,31.426204980645753,31.105880825019693,20.11773030994578,47.37363459438702
G133.5-75.6-294,0.46130268199233715,4.570648166268751e-08,-5.571498219868521,-1,9.893967245596944,True,3.1053847739136637,3.5608394245679476,1.7424588412650028,-6.847658270169869,-16.125002956117964
G248.9+36.8+181,0.08070447918650822,0.452664079053446,-9.22561988896952,-1,1.9355412721916405,False,6.000370649868542,12.326252879593241,10.767175215249209,-1.2534429588517746,-2.9516326048070622
G261.5-34.5+237,0.2889454915944982,5.960798790546759e-08,-9.242870775459728,-1,11.928300395711442,True,8.594854462293316,21.06597665131827,19.23288457423595,6.542074154087057,15.405407354123094
G262.4-26.2+143,0.21329644533869885,1.2224926075128765e-06,5.330447266073939,-1,13.188305123931798,True,10.157119529631,25.64311095033743,23.545744033096597,-28.767857863402234,-67.7431283493408
G264.6-53.3+224,0.3195887445887446,4.0730287570954555e-05,2.861884589161079,1,-6.697674970398481,True,5.927300141806982,15.73809348278103,14.579255793821908,0.35850194470109464,0.8442075655647145


In [6]:
from astropy.table import hstack

bigger_results = hstack([results, wavgs])
big_results_filtered = us.filter_significant(bigger_results)
bigger_results


Name,Statistic,p_value,Statistic_x,Statistic_sgn,Statistic_diff,Significant,Sigma [meas],Sigma [obsv],Sigma [true],Sigma [diff],FWHM,Average [inner],Average [outer],Average [diff],Avg Unc [inner],Avg Unc [outer],Avg Unc [diff]
str15,float64,float64,float64,int32,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
G038.7-37.3-241,0.2074738871132492,3.5194997048780594e-09,-23.129523665567874,-1,16.86046632442212,True,7.315711919790323,21.44495684338161,20.158534994417067,-7.5559356070774975,-17.792868626509186,-9.010901261901203,-16.691518833687702,7.680617571786499,0.1544710566244403,0.06975604516150032,0.16949104156634448
G038.8-33.2-261,0.31066825775656326,1.3943532792837091e-08,-6.963302929232215,-1,12.50616654023725,True,7.247502115395694,22.736827410419288,21.5508012328667,-3.6485845077294528,-8.591759934790694,-0.6960604023293531,-11.978620978240315,11.282560575910962,0.4934626372172539,0.03289730408607167,0.4945579914889038
G043.4-30.4-230,0.24109469074986317,3.606854288675973e-11,-9.607423653882606,-1,17.56975352329457,True,7.411948080584831,26.053745901546854,24.977203629571594,-2.3590579203581576,-5.555156878248415,13.132260146632955,0.8190631739611469,12.313196972671808,0.20749211378340493,0.06703715396383249,0.2180526479863889
G089.0-64.7-311,0.3738140417457306,0.015075834341180974,-17.51573024976091,1,-12.06027479430546,False,4.475548858546416,31.426204980645753,31.105880825019693,20.11773030994578,47.37363459438702,-7.30229848617082,-3.978661824845781,-3.3236366613250388,1.1564596550972988,0.048370593054644674,1.1574707979643466
G133.5-75.6-294,0.46130268199233715,4.570648166268751e-08,-5.571498219868521,-1,9.893967245596944,True,3.1053847739136637,3.5608394245679476,1.7424588412650028,-6.847658270169869,-16.125002956117964,-2.168078601732155,-6.370877536021758,4.202798934289603,0.20314919434020826,0.04004602282589842,0.2070586368767265
G248.9+36.8+181,0.08070447918650822,0.452664079053446,-9.22561988896952,-1,1.9355412721916405,False,6.000370649868542,12.326252879593241,10.767175215249209,-1.2534429588517746,-2.9516326048070622,-3.7630650039131255,-2.4956028506573738,-1.2674621532557517,0.2547830594711783,0.07201239105270783,0.26476440821723396
G261.5-34.5+237,0.2889454915944982,5.960798790546759e-08,-9.242870775459728,-1,11.928300395711442,True,8.594854462293316,21.06597665131827,19.23288457423595,6.542074154087057,15.405407354123094,-11.764841199688588,-17.481018590675696,5.716177390987108,0.561224497952308,0.14982108400037036,0.5808780373820878
G262.4-26.2+143,0.21329644533869885,1.2224926075128765e-06,5.330447266073939,-1,13.188305123931798,True,10.157119529631,25.64311095033743,23.545744033096597,-28.767857863402234,-67.7431283493408,12.994898045135406,2.6170295774063628,10.377868467729042,0.46755894063459996,0.17864583221397004,0.5005254202683129
G264.6-53.3+224,0.3195887445887446,4.0730287570954555e-05,2.861884589161079,1,-6.697674970398481,True,5.927300141806982,15.73809348278103,14.579255793821908,0.35850194470109464,0.8442075655647145,1.6847642877788824,8.566817538848705,-6.882053251069822,0.627066596774432,0.04772657752967211,0.628880229449904


In [7]:
ct.write_processed(bigger_results, "../data_processed/results_weighted_RM")

In [1]:
import sys
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/standard_modules')
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project')

from collation import collator, hvc_snapshot as snap, collation_tools as ct
from synthesis import hvc_looper as hvcl, magnetic_field_derivation as mfd, KStest, uncertainty_subtraction as us

results_post = hvcl.weighted_average_HVCs([], toy_override=True, load_file="../data_processed/toy_model/varsub_results", load_directory="../data_processed/toy_model/toy_hvcs/", save_file="../data_processed/toy_model/wgm_results")

results_post

=== HVC RM LOADER ===
Taking HVC snapshots
Process complete
===HVC WEIGHTED MEAN===
Getting stacked sets
Calculating weighted average


C:\/Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project\synthesis.py:466: RuntimeWarning: divide by zero encountered in divide
  weights = np.nan_to_num(1 / (uncs)**2, nan=0, neginf=0, posinf=0)


Constructing table


Name,Statistic,p_value,Statistic_x,Statistic_sgn,Statistic_diff,Significant,Sigma [meas],Sigma [obsv],Sigma [true],Sigma [diff],FWHM,Average [inner],Average [outer],Average [diff],Avg Unc [inner],Avg Unc [outer],Avg Unc [diff]
str67,float64,float64,float64,int32,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
toy_0_0.0_0.0_0.0.ecsv,0.15512615945713734,0.02416490935579006,0.546875,-1,-0.0036755505505539077,False,0.7181247448032667,0.43199068054162376,nan,nan,nan,0.13206216582583086,0.06187903704760876,0.0701831287782221,0.0019822617659745454,0.0001830740741053514,0.001990697823742734
toy_0_0.0_0.0_0.7853981633974483.ecsv,0.15512615945713734,0.02416490935579006,0.546875,-1,-0.0036755505505539077,False,0.6504944929722889,0.2787985633411827,nan,nan,nan,0.1413938042103961,0.06187903704760876,0.07951476716278734,0.00205210872803258,0.0001830740741053514,0.0020602588061399045
toy_0_0.0_0.0_1.5707963267948966.ecsv,0.12254738650909154,0.1250838416045315,0.109375,-1,0.059324949949946415,False,0.5521426408126524,0.20198135638324727,nan,nan,nan,0.09215149775079459,0.06187903704760876,0.03027246070318583,0.0011376728117382048,0.0001830740741053514,0.0011523087881196794
toy_0_0.0_0.0_2.356194490192345.ecsv,0.0917395802282969,0.4127193027413549,0.3203125,1,-0.13013795045045384,False,0.6252851582222411,0.2553787524570038,nan,nan,nan,0.13185791111151304,0.06187903704760876,0.06997887406390428,0.0020832703328715514,0.0001830740741053514,0.002091298973468948
toy_0_0.0_0.0_3.141592653589793.ecsv,0.10641580894601183,0.2439601275322874,-0.2327370731741204,1,-0.18268702312407392,False,0.6982127086292612,0.4046139519692637,nan,nan,nan,0.1266502050670992,0.06187903704760876,0.06477116801949044,0.002114574047548032,0.0001830740741053514,0.0021224842800768155
toy_0_0.0_0.0_3.9269908169872414.ecsv,0.11772544756360576,0.15422986809584666,0.36328125,1,-0.08716920045045384,False,0.6257551735065523,0.2577992608363467,nan,nan,nan,0.1321422854170526,0.06187903704760876,0.07026324836944385,0.0020904068600567567,0.0001830740741053514,0.002098408196033813
toy_0_0.0_0.0_4.71238898038469.ecsv,0.12254738650909154,0.1250838416045315,0.109375,-1,0.059324949949946415,False,0.5521426408126525,0.20198135638324727,nan,nan,nan,0.09215149775079559,0.06187903704760876,0.03027246070318683,0.0011376728117382176,0.0001830740741053514,0.001152308788119692
toy_0_0.0_0.0_5.497787143782138.ecsv,0.1644719538496607,0.014050395172507146,0.546875,-1,-0.0036755505505539077,False,0.6515736544152392,0.2760786238074689,nan,nan,nan,0.1426047883545859,0.06187903704760876,0.08072575130697714,0.0020562409259283283,0.0001830740741053514,0.002064374690329283
toy_0_0.0_0.7853981633974483_0.0.ecsv,0.12708877627956724,0.10164890612991566,0.546875,-1,-0.0036755505505539077,False,0.6765737106913561,0.35500403496551103,nan,nan,nan,0.14009990953013815,0.06187903704760876,0.07822087248252939,0.0021563223549002325,0.0001830740741053514,0.0021640799927109943
